Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [38]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss + 0.01 * tf.nn.l2_loss(weights))
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [55]:
num_steps = 5000

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 48.156128
Minibatch accuracy: 6.2%
Validation accuracy: 11.6%
Minibatch loss at step 500: 0.759319
Minibatch accuracy: 85.2%
Validation accuracy: 81.5%
Minibatch loss at step 1000: 0.800334
Minibatch accuracy: 80.5%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.566359
Minibatch accuracy: 83.6%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 0.648444
Minibatch accuracy: 88.3%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.782542
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 0.784397
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Minibatch loss at step 3500: 0.774766
Minibatch accuracy: 82.0%
Validation accuracy: 79.8%
Minibatch loss at step 4000: 0.703089
Minibatch accuracy: 82.8%
Validation accuracy: 81.6%
Minibatch loss at step 4500: 0.722478
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Test accuracy: 88.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [62]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss + 0.01 * tf.nn.l2_loss(weights))
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
num_steps = 5000

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  train_subset = train_dataset[0:500, :]
  train_sublabel = train_labels[0:500,:]
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    
    offset = (step * batch_size) % (train_sublabel.shape[0] - batch_size)
    # Generate a minibatch.
    #print(offset,offset+batch_size)
    batch_data = train_subset[offset:(offset + batch_size), :]
    batch_labels = train_sublabel[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 47.814663
Minibatch accuracy: 10.2%
Validation accuracy: 12.4%
Minibatch loss at step 500: 0.603401
Minibatch accuracy: 95.3%
Validation accuracy: 77.5%
Minibatch loss at step 1000: 0.344339
Minibatch accuracy: 98.4%
Validation accuracy: 77.7%
Minibatch loss at step 1500: 0.292235
Minibatch accuracy: 99.2%
Validation accuracy: 77.7%
Minibatch loss at step 2000: 0.278127
Minibatch accuracy: 99.2%
Validation accuracy: 77.9%
Minibatch loss at step 2500: 0.290878
Minibatch accuracy: 99.2%
Validation accuracy: 77.9%
Minibatch loss at step 3000: 0.291162
Minibatch accuracy: 99.2%
Validation accuracy: 78.0%
Minibatch loss at step 3500: 0.280849
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 4000: 0.272476
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 4500: 0.273807
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Test accuracy: 84.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [70]:
batch_size = 128
relu_count = 1024
graph = tf.Graph()
beta = 0.01
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_count]))
  weights_output = tf.Variable(
    tf.truncated_normal([relu_count, num_labels]))
  biases = tf.Variable(tf.zeros([relu_count]))
  biases_output = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  keep_prob = tf.placeholder("float")
  relu_layer_dropout = tf.nn.dropout(hidden_layer, keep_prob)
  
  logits = tf.matmul(relu_layer_dropout, weights_output) + biases_output
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  print(loss.shape)
  regularizers = tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_output)
  loss = tf.reduce_mean(loss + beta * regularizers)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
  tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights)+biases), weights_output) + biases_output)
  test_prediction = tf.nn.softmax(
  tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights)+ biases), weights_output) + biases_output)
num_steps = 6000
train_subset = train_dataset[0:500, :]
train_sublabel = train_labels[0:500,:]
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_sublabel.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_subset[offset:(offset + batch_size), :]
    batch_labels = train_sublabel[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

()
Initialized
Minibatch loss at step 0: 3616.313965
Minibatch accuracy: 7.8%
Validation accuracy: 24.5%
Minibatch loss at step 500: 21.124258
Minibatch accuracy: 99.2%
Validation accuracy: 78.8%
Minibatch loss at step 1000: 0.528277
Minibatch accuracy: 99.2%
Validation accuracy: 79.5%
Minibatch loss at step 1500: 0.327388
Minibatch accuracy: 99.2%
Validation accuracy: 79.5%
Minibatch loss at step 2000: 0.311038
Minibatch accuracy: 99.2%
Validation accuracy: 79.7%
Minibatch loss at step 2500: 0.310143
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 3000: 0.306858
Minibatch accuracy: 99.2%
Validation accuracy: 80.1%
Minibatch loss at step 3500: 0.294649
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 4000: 0.303548
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Minibatch loss at step 4500: 0.290039
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 5000: 0.294559
Minibatch accuracy: 100.0%
Validat

# ---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [47]:
import math as math
import random as random
batch_size = 128
relu_count = 512
factor = 0.8
graph = tf.Graph()
beta = 0.001

def getlogits(dataset, weights, biases, dropout, keep_prob):
 
  print(keep_prob)
  logits = dataset
  for i in range(count):
    logits = tf.matmul(logits, weights[i]) + biases[i]
    if i < count - 1:
      logits = tf.nn.relu(logits)
      if dropout:
        logits = tf.nn.dropout(logits, keep_prob)
  return logits
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  wcounts = [image_size * image_size]
  keep_prob = tf.placeholder("float")

  count = 8
  for i in range(count-1):
    wcounts.append((int)(relu_count * math.pow(factor, i)))
  wcounts.append(num_labels)
  print(wcounts)
  weights = np.array([])
  biases = np.array([])
  for i in range(count):
    weights = np.append(weights, tf.Variable(
    tf.truncated_normal([wcounts[i], wcounts[i+1]], stddev=math.sqrt(2.0/(wcounts[i])))))
    biases = np.append(biases, tf.Variable(tf.zeros([wcounts[i+1]])))
  # Variables.
  logits = getlogits(tf_train_dataset, weights, biases, True, 0.5)
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  regularizers = 0
  for i in range(count):
    regularizers = regularizers + tf.nn.l2_loss(weights[i])
  loss = tf.reduce_mean(loss + beta * regularizers)

  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(getlogits(tf_valid_dataset, weights, biases, False, None))
  test_prediction = tf.nn.softmax(getlogits(tf_test_dataset, weights, biases, False, None))
num_steps = 10000
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = random.randint(0, train_labels.shape[0] - batch_size -1)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

[784, 512, 409, 327, 262, 209, 167, 134, 10]
0.5
None
None
Initialized
Minibatch loss at step 0: 5.583579
Minibatch accuracy: 9.4%
Validation accuracy: 9.7%
Minibatch loss at step 500: 2.579970
Minibatch accuracy: 41.4%
Validation accuracy: 33.1%
Minibatch loss at step 1000: 1.916743
Minibatch accuracy: 57.8%
Validation accuracy: 68.6%
Minibatch loss at step 1500: 1.237257
Minibatch accuracy: 77.3%
Validation accuracy: 74.4%
Minibatch loss at step 2000: 1.098326
Minibatch accuracy: 73.4%
Validation accuracy: 75.7%
Minibatch loss at step 2500: 1.184225
Minibatch accuracy: 72.7%
Validation accuracy: 77.6%
Minibatch loss at step 3000: 1.130402
Minibatch accuracy: 79.7%
Validation accuracy: 83.1%
Minibatch loss at step 3500: 1.185547
Minibatch accuracy: 70.3%
Validation accuracy: 84.2%
Minibatch loss at step 4000: 0.757034
Minibatch accuracy: 85.9%
Validation accuracy: 84.2%
Minibatch loss at step 4500: 0.846960
Minibatch accuracy: 83.6%
Validation accuracy: 84.4%
Minibatch loss at step 50